In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/tracking/sot/

!pip install torch
!pip install opencv-python
!pip install --upgrade git+https://github.com/got-10k/toolkit.git@master

/content/drive/MyDrive/tracking/sot
  Cloning https://github.com/got-10k/toolkit.git (to revision master) to /tmp/pip-req-build-576teh99
  Running command git clone --filter=blob:none --quiet https://github.com/got-10k/toolkit.git /tmp/pip-req-build-576teh99
  Resolved https://github.com/got-10k/toolkit.git to commit 956e7286fdf209cbb125adac9a46376bd8297ffb
  Preparing metadata (setup.py) ... done


In [4]:
!pwd
gt_bbox = [298, 160, 48, 80]

/content/drive/MyDrive/tracking/sot


In [5]:
import sys
sys.path.append('/content/drive/My Drive/tracking/sot/')

In [6]:
import cv2
from siamrpn import TrackerSiamRPN
import os


In [7]:
# تنظیم مسیر مدل
model_path = 'siamrpn.pth'
assert os.path.exists(model_path), f'Model not found at {model_path}'

# بارگذاری مدل
tracker = TrackerSiamRPN(net_path=model_path)

# بارگذاری ویدیو
video_path = 'Coke.mp4'

In [8]:

cap = cv2.VideoCapture(video_path)
ret, first_frame = cap.read()
if not ret:
    raise Exception("خطا در خواندن فریم اول")

# 🎯 مختصات عددی شیء (مثلاً دستی داده شده)
init_bbox = (298, 160, 48, 80)  # (x, y, w, h)

# مقداردهی اولیه ردیاب
tracker.init(first_frame, init_bbox)

# ذخیره ویدیوی خروجی
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.avi', fourcc, fps, (w, h))

# پردازش بقیه فریم‌ها
while True:
    ret, frame = cap.read()
    if not ret:
        break

    bbox = tracker.update(frame)
    x, y, w, h = [int(i) for i in bbox]
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
